# Using Spark right away ... ensuring scalability
- Spark seems to be an overkill for simple projects.
- However, small projects often very quickly outgrow the laptop.
- Avoid the pain of scaling up.

## Why Spark?
1. Spark operates on Resilent Distributed Datasets (RDDs) that can be used to represent data tables.
2. The same code for local environment (laptop) and cluster.
3. Spark's concept of "transformations" (e.g. "map") and "actions" (e.g. "reduce") makes it easy to wirte parallel programs.
4. A lot of the ETL work can be taken care of in fairly intuitive ways.
4. Spark + Hive supports SQL queries on data tables. Sometimes, SQL offers the most elegant way.
5. Spark has it's own machine learning library
6. In local mode Spark reads from regular file system; on the cluster it usually reads from HDFS.

## We need Java...
Install the latest JDK (... I use Oracle, old habit)

## Spark Installation
Use HomeBrew on OS X or package manager for Linux
<pre>
$ brew install apache-spark
</pre>

Try it out:
<pre>
$ pyspark
...
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.2
      /_/

Using Python version 2.7.12 (default, Jul  2 2016 17:43:17)
SparkContext available as sc, HiveContext available as sqlContext.
>>>
</pre>

Spark used Python 2 by default...hence, the driver program should be running Python 2 as well...

## Jupyter Notebook
This makes it a bit more complicated. There are tons of post of the Internet, but they seem to be obsolete in about 17 days on average....

Here are some fairly pedestrian steps:
1. Create a Python 2 environment using anaconda, and activate since we need to install things...
<pre>
$ conda create -n python2 python=2.7 anaconda
$ source  activate python2
</pre>
2. Install a magic tool called "findspark"...
<pre>
$ pip install findspark
</pre>

The Jupyter Notebook shoud now offer an additional kernel for this environment.

## Install JDBC
While we're at it, let's download a JDBC driver and store it somewhere. Mine is at
<pre>
/opt/sqljdbc_4.0/enu/sqljdbc4.jar
</pre>

# Get it running ...
While the <code>pyspark</code> takes care of all the prep work we need to do some of it manually in the notebook.

In [1]:
import platform
platform.python_version()

'2.7.12'

In [4]:
import os
import sys

###os.environ['SPARK_PREPEND_CLASSES']='/opt/sqljdbc_4.0/enu/sqljdbc4.jar'
os.environ['SPARK_CLASSPATH']='/opt/sqljdbc_4.0/enu/sqljdbc4.jar'
import findspark

findspark.init()

import py4j
import pyspark
from pyspark.sql import SQLContext, Row
from pyspark import SparkConf, SparkContext

The following lines define where to run Spark:

In [ ]:
conf = SparkConf().setMaster("local").setAppName("My App")
# sc = SparkContext()
sc = SparkContext(conf = conf)

In [6]:
sc.version

u'1.6.2'

In [7]:
query = """(SELECT top 100
	p.ParseID
	,c.Measure AS City
	,s.Measure AS State
FROM
	(
	SELECT
		ParseID
	FROM
		dbo.ParsedData
	WHERE
			ProfileID = 84
		AND MeasureType = 'position'
		AND Measure = 'search_input'
	) AS p

	INNER JOIN	(
				SELECT
					ParseID
					,Measure
				FROM
					dbo.ParsedData
				WHERE
					MeasureType = 'city'
				) AS c ON
		p.ParseID = c.ParseID

	INNER JOIN	(
				SELECT
					ParseID
					,Measure
				FROM
					dbo.ParsedData
				WHERE
					MeasureType = 'state'
				) AS s ON
		p.ParseID = s.ParseID) as pd
"""


In [8]:
sqlctx = SQLContext(sc)
df = sqlctx.read.jdbc("jdbc:sqlserver://warehouse:1433;databaseName=Staging;user=***;password=***", query)

In [31]:
print df.printSchema()

root
 |-- ParseID: long (nullable = true)
 |-- City: string (nullable = false)
 |-- State: string (nullable = false)

None


In [10]:
df.toPandas()

,ParseID,City,State
0,20160719000046278,Winter-Haven,Florida
1,20160719000052396,Texas-State-University---San-Marcos,Texas
2,20160719000062181,Berkeley,California
3,20160719000166357,Portland,Oregon
4,20160719000288190,Salt-Lake-City,Utah
5,20160719000297070,Glenside,Pennsylvania
6,20160719000001923,Edgerton,Wisconsin
7,20160719000002925,Apple-Valley,Minnesota
8,20160719000019371,Independence,Missouri
9,20160719000040670,Orlando,Florida


# Twitter
Let's work on some Twitter data...just a small set of 1,000 records, have 170,000,000

In [15]:
tweets = sc.textFile('cache-0-json_first1k.txt')
tweets.take(1)

[u'{"text":"Obama vies for health care edge in Florida - http:\\/\\/t.co\\/OcISvreb http:\\/\\/t.co\\/FsJ7xgGW #florida","in_reply_to_user_id":null,"truncated":false,"id_str":"244907511377965056","retweeted":false,"in_reply_to_status_id_str":null,"source":"<a href=\\"http:\\/\\/twitterfeed.com\\" rel=\\"nofollow\\">twitterfeed<\\/a>","possibly_sensitive":false,"entities":{"hashtags":[{"text":"florida","indices":[87,95]}],"urls":[{"indices":[45,65],"display_url":"STLtoday.com","expanded_url":"http:\\/\\/STLtoday.com","url":"http:\\/\\/t.co\\/OcISvreb"},{"indices":[66,86],"display_url":"bit.ly\\/Tzhmly","expanded_url":"http:\\/\\/bit.ly\\/Tzhmly","url":"http:\\/\\/t.co\\/FsJ7xgGW"}],"user_mentions":[]},"in_reply_to_screen_name":null,"in_reply_to_status_id":null,"in_reply_to_user_id_str":null,"place":null,"contributors":null,"coordinates":null,"created_at":"Sun Sep 09 21:17:55 +0000 2012","favorited":false,"possibly_sensitive_editable":true,"user":{"follow_request_sent":null,"contributors

In [16]:
def extract_hashtags(tw):
    import json
    return json.loads(tw)['entities']['hashtags']

hashtags = tweets.map(extract_hashtags)
hashtags.take(5)

[[{u'indices': [87, 95], u'text': u'florida'}],
 [{u'indices': [88, 92], u'text': u'USA'},
  {u'indices': [93, 101], u'text': u'cottage'},
  {u'indices': [102, 110], u'text': u'awesome'}],
 [{u'indices': [104, 112], u'text': u'florida'}],
 [],
 []]

In [18]:
hashtags = tweets.flatMap(extract_hashtags)
hashtags.take(5)

[{u'indices': [87, 95], u'text': u'florida'},
 {u'indices': [88, 92], u'text': u'USA'},
 {u'indices': [93, 101], u'text': u'cottage'},
 {u'indices': [102, 110], u'text': u'awesome'},
 {u'indices': [104, 112], u'text': u'florida'}]

In [19]:
tuples = hashtags.map(lambda x: (x['text'], 1))
tuples.take(5)

[(u'florida', 1),
 (u'USA', 1),
 (u'cottage', 1),
 (u'awesome', 1),
 (u'florida', 1)]

In [20]:
counts = tuples.reduceByKey(lambda a, b: a+b)
counts.take(5)

[(u'SQUADD', 1),
 (u'ObamaInFL', 1),
 (u'Collegiate', 1),
 (u'september', 1),
 (u'money', 1)]

In [23]:
topcounts = counts.map(lambda t: (t[1], t[0])).sortByKey(False)
topcounts.take(10)

[(18, u'USA'),
 (14, u'Obama'),
 (12, u'tcot'),
 (6, u'p2'),
 (5, u'ThingsNobodyWouldSay'),
 (5, u'obama'),
 (4, u'IFWT'),
 (3, u'Sports'),
 (3, u'election'),
 (3, u'Amazon')]

In [27]:
topcntdf = sqlctx.createDataFrame(topcounts)
topcntdf.take(5)

[Row(_1=18, _2=u'USA'),
 Row(_1=14, _2=u'Obama'),
 Row(_1=12, _2=u'tcot'),
 Row(_1=6, _2=u'p2'),
 Row(_1=5, _2=u'ThingsNobodyWouldSay')]

In [30]:
topcntdf.printSchema()

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)



In [28]:
topcntdf.toPandas()

,_1,_2
0,18,USA
1,14,Obama
2,12,tcot
3,6,p2
4,5,ThingsNobodyWouldSay
5,5,obama
6,4,IFWT
7,3,Sports
8,3,election
9,3,Amazon
